# CNN Model - Previous Fire Data and Weather

This notebook takes data from S3, trains a CNN model using Keras and Tensorflow, and saves the model to S3.

In [19]:
# Load packages
import boto3
import csv
import io
import json
import math
import numpy as np
import os
import pandas as pd
import pickle
import random

from datetime import datetime as dt
from keras.models import model_from_json
from matplotlib import pyplot as plt
from PIL import Image

### Variables and Hyperparameters

In [20]:
# s3 config
s3_client = boto3.client('s3')
bucket_name = 'hotzone'

# CNN config

# the desired height and width (in pixels) of the matrix to feed into the CNN
# 1 pixel side = 500 meters = 0.310686 miles
matrix_dim = 32

# test size for train/test split
test_size = 0.2

# training epochs
epoc = 10

## Pull Data from S3

In [21]:
def pull_data_from_s3(s3_client, bucket_name, key_name):
    '''
    Pulls pre-processed data from S3.

    Args:
        - s3_client: boto3 s3 client
        - bucket_name: name of bucket on s3 to pull data from
        - key_name: directory/file_name to pull data from
    Returns:
        - Nothing
    
    https://stackoverflow.com/questions/48049557/how-to-write-npy-file-to-s3-directly
    '''
    
    array_data = io.BytesIO()
    s3_client.download_fileobj(bucket_name, key_name, array_data)
    
    array_data.seek(0)
    array = pickle.load(array_data)

    return array

In [22]:
years = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

fire = []
weather = []
Y = []

for y in years:
    print('*****************************************')
    print('Starting data preprocessing for year: ', y)
    time = dt.now()
    print (time)
    print('*****************************************')
    
    fire_key_name = "input_fire/fire_{}.pickle".format(str(y))
    weather_key_name = "input_weather/weather_{}.pickle".format(str(y))
    label_key_name = "labels/label_{}.pickle".format(str(y))
    
    fire_data = pull_data_from_s3(s3_client, bucket_name, fire_key_name)
    weather_data = pull_data_from_s3(s3_client, bucket_name, weather_key_name)
    labels = pull_data_from_s3(s3_client, bucket_name, label_key_name)
    
    fire.append(fire_data)
    weather.append(weather_data)
    Y.append(labels)

*****************************************
Starting data preprocessing for year:  2003
2020-04-10 18:50:24.198759
*****************************************
*****************************************
Starting data preprocessing for year:  2004
2020-04-10 18:50:26.648937
*****************************************
*****************************************
Starting data preprocessing for year:  2005
2020-04-10 18:50:29.383776
*****************************************
*****************************************
Starting data preprocessing for year:  2006
2020-04-10 18:50:30.326074
*****************************************
*****************************************
Starting data preprocessing for year:  2007
2020-04-10 18:50:32.172811
*****************************************
*****************************************
Starting data preprocessing for year:  2008
2020-04-10 18:50:33.752327
*****************************************
*****************************************
Starting data preprocessing 

In [23]:
fire = np.concatenate(fire)
weather = np.concatenate(weather)
Y = np.concatenate(Y)

print(fire.shape)
print(weather.shape)
print(Y.shape)

(656500, 32, 32, 1)
(656500, 8)
(656500,)


## Build CNN

In [24]:
# import packages

from __future__ import print_function

import tensorflow as tf

import keras
import keras.backend as K

from keras.models import Sequential, Model
from keras.layers import AveragePooling2D, Conv1D, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input, concatenate

In [25]:
# compute f1 score manually - taken from https://datascience.stackexchange.com/a/45166

def recall_m(y_true, y_pred):
    '''
    Computes recall.
    
    Args:
        - y_true: true values of target variable.
        - y_pred: predicted values of target variable.
    Returns:
        - recall: true positives / actual results
    '''
    
    true_pos = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_pos = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_pos / (possible_pos + K.epsilon())

    return recall


def precision_m(y_true, y_pred):
    '''
    Computes precision.
    
    Args:
        - y_true: true values of target variable.
        - y_pred: predicted values of target variable.
    Returns:
        - precision: true positives / predicted results
    '''
    
    true_pos = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_pos = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_pos / (predicted_pos + K.epsilon())
    
    return precision


def f1_score(y_true, y_pred):
    '''
    Args:
        - y_true: true values of target variable.
        - y_pred: predicted values of target variable.
    Returns:
        - score: f1 score
    '''
    
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    score = 2*((precision*recall)/(precision+recall+K.epsilon()))
    
    return score

In [26]:
# Create model_2: image data, weather data, and fire speed/direction data with functional API

# Define image inputs shape
image_shape = fire[0].shape
image_inputs = Input(shape = image_shape)

# Define weather inputs shape
weather_shape = weather[0].shape
weather_inputs = Input(shape = weather_shape)

# Add layers for fire image interpretation
fire_1 = AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid')(image_inputs)

fire_2 = Conv2D(32, kernel_size=(3, 3), activation='sigmoid')(fire_1)
fire_3 = MaxPooling2D(pool_size=(2,2), strides=None, padding='valid')(fire_2)
fire_4 = Dropout(0.2)(fire_3)

fire_5 = Conv2D(64, kernel_size=(3, 3), activation='sigmoid')(fire_4)
fire_6 = MaxPooling2D(pool_size=(2,2), strides=None, padding='valid')(fire_5)
fire_7 = Dropout(0.2)(fire_6)

fire_8 = Flatten()(fire_7)
fire_9 = Dense(128, activation='sigmoid')(fire_8)

# Combine the layers
concat = concatenate([fire_9, weather_inputs])

# Final dense layer 
predictions = Dense(1, activation='sigmoid')(concat)

# Define the model
model_2 = Model(inputs=[image_inputs, weather_inputs], outputs=predictions)

In [27]:
%%time
# compile the model
model_2.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy', f1_score, tf.keras.metrics.AUC()]
)

CPU times: user 144 ms, sys: 0 ns, total: 144 ms
Wall time: 144 ms


In [29]:
%%time
# fit the model
model_2.fit(
    x = [fire, weather], 
    y = Y,
    validation_split = test_size, 
    epochs=epoc
)

Train on 525200 samples, validate on 131300 samples
Epoch 1/10
525200/525200 [==============================] - 271s 516us/step - loss: 0.0306 - accuracy: 0.9929 - f1_score: 0.6144 - auc_1: 0.9179 - val_loss: 0.0088 - val_accuracy: 0.9984 - val_f1_score: 0.7026 - val_auc_1: 0.9822
Epoch 2/10
525200/525200 [==============================] - 266s 507us/step - loss: 0.0094 - accuracy: 0.9982 - f1_score: 0.7023 - auc_1: 0.9868 - val_loss: 0.0083 - val_accuracy: 0.9986 - val_f1_score: 0.7065 - val_auc_1: 0.9886
Epoch 3/10
525200/525200 [==============================] - 265s 505us/step - loss: 0.0080 - accuracy: 0.9984 - f1_score: 0.7095 - auc_1: 0.9899 - val_loss: 0.0084 - val_accuracy: 0.9986 - val_f1_score: 0.7076 - val_auc_1: 0.9904
Epoch 4/10
525200/525200 [==============================] - 266s 506us/step - loss: 0.0069 - accuracy: 0.9987 - f1_score: 0.7170 - auc_1: 0.9910 - val_loss: 0.0080 - val_accuracy: 0.9985 - val_f1_score: 0.7077 - val_auc_1: 0.9913
Epoch 5/10
525200/525200 [==

## Save CNN to S3

In [30]:
def save_array_to_s3(s3_client, array, bucket_name, key_name):
    '''
    Uploads pre-processed data to S3.

    Args:
        - s3_client: boto3 s3 client
        - array: numpy array to save to s3
        - bucket_name: name of bucket on s3 to save array to
        - key_name: directory/file_name to save data to
    Returns:
        - Nothing
    
    https://stackoverflow.com/questions/48049557/how-to-write-npy-file-to-s3-directly
    '''
    
    array_data = io.BytesIO()
    pickle.dump(array, array_data)
    array_data.seek(0)
    
    s3_client.upload_fileobj(array_data, bucket_name, key_name)

In [31]:
# get model config and model weights

config = model_2.get_config()
weights = model_2.get_weights()

In [32]:
# save model config and model weights to s3

save_array_to_s3(s3_client, config, bucket_name, 'models/model_config.pickle')
save_array_to_s3(s3_client, weights, bucket_name, 'models/model_weights.pickle')

## Load CNN from S3

In [14]:
new_config = pull_data_from_s3(s3_client, bucket_name, 'models/model_config.pickle')
new_weights = pull_data_from_s3(s3_client, bucket_name, 'models/model_weights.pickle')


new_model = keras.Model.from_config(new_config)
new_model.set_weights(new_weights)